# Lab 10: Integration Project - Capstone

## Project Overview

In this final lab, you'll build a comprehensive project that combines all concepts learned throughout the course:
- Variables and data types
- Control flow (loops, conditionals)
- Functions and modularity
- Data structures
- String operations
- File I/O and error handling
- Object-oriented programming
- Data analysis with libraries

---

## Project: Student Grade Management System

### Project Requirements

Build a comprehensive system to manage student grades with the following features:

1. **Data Storage**: Store student information (name, ID, grades in multiple subjects)
2. **CRUD Operations**: Create, read, update, delete student records
3. **Analysis**: Calculate statistics (average, median, pass/fail)
4. **File I/O**: Save and load data from CSV files
5. **Reporting**: Generate performance reports
6. **Error Handling**: Handle invalid inputs gracefully

---

## Phase 1: Data Structures and Classes

import pandas as pd
import numpy as np
from datetime import datetime

# Define a Student class
class Student:
    """Represents a student with their information and grades."""
    
    def __init__(self, name, student_id, grades=None):
        """
        Initialize a student.
        
        Args:
            name (str): Student's name
            student_id (str): Unique student ID
            grades (dict): Dictionary of subject -> grade
        """
        self.name = name
        self.student_id = student_id
        self.grades = grades if grades else {}
    
    def add_grade(self, subject, grade):
        """Add a grade for a subject."""
        if not 0 <= grade <= 100:
            raise ValueError(f"Grade must be between 0 and 100, got {grade}")
        self.grades[subject] = grade
    
    def get_average(self):
        """Calculate average grade."""
        if not self.grades:
            return 0
        return sum(self.grades.values()) / len(self.grades)
    
    def get_status(self, passing_grade=60):
        """Get pass/fail status."""
        average = self.get_average()
        return "Pass" if average >= passing_grade else "Fail"
    
    def __str__(self):
        return f"Student: {self.name} (ID: {self.student_id}), Avg: {self.get_average():.2f}"

# Test the Student class
student1 = Student("Alice", "S001")
student1.add_grade("Math", 85)
student1.add_grade("English", 90)
student1.add_grade("Science", 78)

print(student1)
print(f"Average: {student1.get_average():.2f}")
print(f"Status: {student1.get_status()}")

## Phase 2: Grade Management Class

class GradeManager:
    """Manages a collection of students and their grades."""
    
    def __init__(self):
        self.students = {}
    
    def add_student(self, name, student_id):
        """Add a new student."""
        if student_id in self.students:
            raise ValueError(f"Student ID {student_id} already exists")
        self.students[student_id] = Student(name, student_id)
        return self.students[student_id]
    
    def get_student(self, student_id):
        """Retrieve a student by ID."""
        if student_id not in self.students:
            raise ValueError(f"Student ID {student_id} not found")
        return self.students[student_id]
    
    def add_grade_to_student(self, student_id, subject, grade):
        """Add a grade to a student."""
        student = self.get_student(student_id)
        student.add_grade(subject, grade)
    
    def get_class_statistics(self):
        """Get class-wide statistics."""
        if not self.students:
            return None
        
        averages = [s.get_average() for s in self.students.values()]
        
        return {
            'mean': np.mean(averages),
            'median': np.median(averages),
            'std': np.std(averages),
            'min': np.min(averages),
            'max': np.max(averages),
            'total_students': len(self.students)
        }
    
    def get_passing_students(self, passing_grade=60):
        """Get students who passed."""
        return [s for s in self.students.values() 
                if s.get_average() >= passing_grade]
    
    def to_dataframe(self):
        """Convert to pandas DataFrame."""
        data = []
        for sid, student in self.students.items():
            data.append({
                'ID': sid,
                'Name': student.name,
                'Average': student.get_average(),
                'Status': student.get_status()
            })
        return pd.DataFrame(data)

# Test the GradeManager
manager = GradeManager()

# Add students
manager.add_student("Alice", "S001")
manager.add_student("Bob", "S002")
manager.add_student("Charlie", "S003")

# Add grades
manager.add_grade_to_student("S001", "Math", 85)
manager.add_grade_to_student("S001", "English", 90)
manager.add_grade_to_student("S002", "Math", 75)
manager.add_grade_to_student("S002", "English", 80)
manager.add_grade_to_student("S003", "Math", 55)
manager.add_grade_to_student("S003", "English", 58)

# Display results
print("Class Statistics:")
stats = manager.get_class_statistics()
for key, value in stats.items():
    print(f"  {key}: {value:.2f}")

print("\nDataFrame:")
print(manager.to_dataframe())

## Phase 3: File I/O with Error Handling

def save_to_csv(manager, filename):
    """Save grade data to CSV."""
    try:
        df = manager.to_dataframe()
        df.to_csv(filename, index=False)
        print(f"Successfully saved to {filename}")
    except IOError as e:
        print(f"Error saving file: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

def load_from_csv(filename):
    """Load student data from CSV."""
    try:
        df = pd.read_csv(filename)
        print(f"Successfully loaded from {filename}")
        return df
    except FileNotFoundError:
        print(f"File {filename} not found")
        return None
    except pd.errors.EmptyDataError:
        print(f"File {filename} is empty")
        return None
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# Test file operations
save_to_csv(manager, "grades.csv")
loaded_df = load_from_csv("grades.csv")
if loaded_df is not None:
    print("\nLoaded data:")
    print(loaded_df)

## Phase 4: Reporting and Analysis

def generate_report(manager):
    """Generate a comprehensive grade report."""
    print("=" * 50)
    print("GRADE REPORT".center(50))
    print("=" * 50)
    print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    # Class statistics
    stats = manager.get_class_statistics()
    print("CLASS STATISTICS:")
    print(f"  Total Students: {int(stats['total_students'])}")
    print(f"  Class Average: {stats['mean']:.2f}")
    print(f"  Median Grade: {stats['median']:.2f}")
    print(f"  Std Dev: {stats['std']:.2f}")
    print(f"  Highest: {stats['max']:.2f}")
    print(f"  Lowest: {stats['min']:.2f}")
    print()
    
    # Student details
    print("STUDENT DETAILS:")
    df = manager.to_dataframe()
    df_sorted = df.sort_values('Average', ascending=False)
    for idx, row in df_sorted.iterrows():
        print(f"  {row['Name']:15} (ID: {row['ID']}) - "
              f"Avg: {row['Average']:6.2f} - {row['Status']}")
    print()
    
    # Passing rate
    passing = len(manager.get_passing_students())
    total = stats['total_students']
    pass_rate = (passing / total * 100) if total > 0 else 0
    print(f"Pass Rate: {passing}/{int(total)} ({pass_rate:.1f}%)")
    print("=" * 50)

# Generate and display report
generate_report(manager)

## Phase 5: Interactive Menu System

def display_menu():
    """Display the main menu."""
    print("\n" + "="*40)
    print("GRADE MANAGEMENT SYSTEM".center(40))
    print("="*40)
    print("1. Add Student")
    print("2. Add Grade")
    print("3. View Student Info")
    print("4. View Class Report")
    print("5. Save to File")
    print("6. List All Students")
    print("7. Exit")
    print("="*40)
    return input("Choose an option (1-7): ")

def interactive_session():
    """Run an interactive session."""
    manager = GradeManager()
    
    # Pre-populate with sample data
    print("Loading sample data...")
    manager.add_student("Alice Johnson", "S001")
    manager.add_student("Bob Smith", "S002")
    manager.add_student("Charlie Brown", "S003")
    
    manager.add_grade_to_student("S001", "Math", 88)
    manager.add_grade_to_student("S001", "English", 92)
    manager.add_grade_to_student("S002", "Math", 76)
    manager.add_grade_to_student("S002", "English", 82)
    manager.add_grade_to_student("S003", "Math", 65)
    manager.add_grade_to_student("S003", "English", 70)
    
    while True:
        choice = display_menu()
        
        if choice == '1':
            try:
                name = input("Enter student name: ")
                sid = input("Enter student ID: ")
                manager.add_student(name, sid)
                print(f"✓ Student {name} added successfully")
            except ValueError as e:
                print(f"✗ Error: {e}")
        
        elif choice == '2':
            try:
                sid = input("Enter student ID: ")
                subject = input("Enter subject: ")
                grade = float(input("Enter grade: "))
                manager.add_grade_to_student(sid, subject, grade)
                print(f"✓ Grade added successfully")
            except ValueError as e:
                print(f"✗ Error: {e}")
        
        elif choice == '3':
            try:
                sid = input("Enter student ID: ")
                student = manager.get_student(sid)
                print(f"\n{student}")
                print(f"Grades: {student.grades}")
            except ValueError as e:
                print(f"✗ Error: {e}")
        
        elif choice == '4':
            generate_report(manager)
        
        elif choice == '5':
            save_to_csv(manager, "grades.csv")
        
        elif choice == '6':
            df = manager.to_dataframe()
            print("\n" + df.to_string(index=False))
        
        elif choice == '7':
            print("Thank you for using Grade Management System!")
            break
        
        else:
            print("✗ Invalid option. Please choose 1-7")

# Uncomment to run interactive session
# interactive_session()

# For demonstration, show the system in action
print("Grade Management System is ready!")
print("To run the interactive session, uncomment the last line.")

---

## Project Tasks

### Task 1: Extend the System
Add the following features:
1. Delete student functionality
2. Update student grades
3. Calculate weighted grades (if subjects have different weights)
4. Grade distribution histogram

# TODO: Task 1 - Extend the system
# Add your extensions here:



### Task 2: Data Validation
Improve error handling with:
1. Validate student names (no empty strings)
2. Validate student IDs (unique, proper format)
3. Validate grades (0-100 range)
4. Handle duplicate grade entries

# TODO: Task 2 - Data validation
# Add validation functions here:



### Task 3: Advanced Analytics
Add analysis features:
1. Grade improvement tracking
2. Subject-wise performance analysis
3. Identify struggling students
4. Recommend interventions based on grades

# TODO: Task 3 - Advanced analytics
# Add analytics functions here:



### Task 4: Visualization
Create visual representations (requires matplotlib):
1. Bar chart of student averages
2. Histogram of grade distribution
3. Pie chart of pass/fail rates

Note: Install matplotlib if needed: pip install matplotlib

# TODO: Task 4 - Visualization
# Add visualization functions here:
# import matplotlib.pyplot as plt



### Task 5: Testing Your System
Test the complete system:
1. Create multiple students
2. Add various grades
3. Generate reports
4. Save and load data
5. Test error handling with invalid inputs

# TODO: Task 5 - Comprehensive testing
# Test your enhanced system here:



---

## Reflection Questions

After completing this project, answer the following:

1. **Object-Oriented Design**: How did using classes improve code organization?
2. **Error Handling**: What types of errors did you encounter and how did you handle them?
3. **Data Processing**: How did pandas help with data manipulation and analysis?
4. **File I/O**: What challenges did you face with reading/writing files?
5. **Future Enhancements**: What features would you add to make this system more useful?
6. **Learning Outcomes**: How has your Python programming skill level improved?

# TODO: Reflection
# Write your reflections here:
"""
1. Object-Oriented Design: 
   

2. Error Handling:
   

3. Data Processing:
   

4. File I/O:
   

5. Future Enhancements:
   

6. Learning Outcomes:
   
"""

---

## Course Completion Summary

### Congratulations!

You have successfully completed the Python Programming Series!

### Topics Covered:

1. **Lab 1**: Python Basics - Variables, data types, operators
2. **Lab 2**: Control Flow - Conditionals and loops
3. **Lab 3**: Functions - Modularity and code reuse
4. **Lab 4**: Data Structures - Lists, tuples, dictionaries, sets
5. **Lab 5**: String Operations - Text processing and formatting
6. **Lab 6**: File I/O - Reading, writing, and error handling
7. **Lab 7**: Advanced Functions - Comprehensions and lambda
8. **Lab 8**: OOP - Classes, inheritance, and polymorphism
9. **Lab 9**: Libraries - NumPy and Pandas for data analysis
10. **Lab 10**: Integration - Real-world project combining all concepts

### Skills Achieved:
- ✅ Intermediate Python Programming
- ✅ Object-Oriented Programming
- ✅ Data Processing and Analysis
- ✅ File and Error Management
- ✅ Working with External Libraries
- ✅ Building Real Applications

### Next Steps:
- Explore advanced Python topics (decorators, generators, async programming)
- Learn data visualization (Matplotlib, Seaborn, Plotly)
- Study web development (Flask, Django)
- Explore machine learning (Scikit-learn, TensorFlow)
- Contribute to open-source projects
- Build personal projects and portfolio

### Resources for Continued Learning:
- Official Python Documentation: https://docs.python.org/3/
- Python Enhancement Proposals (PEPs): https://peps.python.org/
- Real Python: https://realpython.com/
- Stack Overflow: https://stackoverflow.com/
- GitHub: https://github.com/ (for open-source contributions)

---

**Thank you for completing this Python programming course!**

*Happy coding!* 🐍

---

## 🚀 Open in Google Colab

Click the button below to run this notebook in Google Colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AhmadAlsaadi/ENCH320-Labs/blob/main/Notebook_10_Integration_Project.ipynb)

---

## 🎓 Integration Project Summary

You have successfully completed:
- ✅ **Project Planning** - Breaking down complex problems
- ✅ **Integration** - Using multiple Python concepts together
- ✅ **Real-World Application** - Solving practical problems
- ✅ **Code Organization** - Writing clean, maintainable code
- ✅ **Problem Solving** - Debugging and testing your solution
- ✅ **Complete Curriculum** - Mastering Python basics to intermediate

### Congratulations!
You've completed the entire Python curriculum. You now have the skills to:
- Write Python programs for real-world applications
- Work with data structures and libraries
- Organize code using functions and classes
- Handle errors and edge cases
- Continue learning advanced Python topics

---

<details>
<summary style="cursor: pointer; color: #667EEA; font-weight: bold; font-size: 14px; font-family: 'Amiri', serif;">🌍 Arabic Translation / الترجمة العربية</summary>

<div dir="rtl" style="text-align: right; margin-top: 10px; padding: 15px; background: linear-gradient(135deg, #F5F5F5 0%, #FAFAFA 100%); border-radius: 8px; border-right: 4px solid #667EEA; font-family: 'Amiri', serif; font-size: 16px; line-height: 1.8;">

<link href="https://fonts.googleapis.com/css2?family=Amiri:wght@400;700&display=swap" rel="stylesheet">

---

## 🎓 ملخص مشروع التكامل

لقد أكملت بنجاح:
- ✅ **التخطيط للمشروع** - تقسيم المشاكل المعقدة
- ✅ **التكامل** - استخدام مفاهيم Python متعددة معاً
- ✅ **التطبيق العملي** - حل المشاكل العملية
- ✅ **تنظيم الكود** - كتابة رمز نظيف وسهل الصيانة
- ✅ **حل المشاكل** - تصحيح وتجربة الحل الخاص بك
- ✅ **المنهج الكامل** - إتقان Python من الأساسيات إلى المستوى المتوسط

### مبروك!
لقد أكملت منهج Python بالكامل. لديك الآن المهارات:
- كتابة برامج Python للتطبيقات الحقيقية
- العمل مع هياكل البيانات والمكتبات
- تنظيم الكود باستخدام الدوال والفئات
- التعامل مع الأخطاء والحالات الحدية
- الاستمرار في تعلم مواضيع Python المتقدمة

</div>
</details>